In [ ]:
# Install relevant packages
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs
!pip install seqeval

In [ ]:
# Link to Hugging face for model checkpoint saving
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# Mount from drive to read the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# create two dictionaries: labels to indices, and indices to labels
import json
bio_list =set()

with open('/content/drive/MyDrive/Final_Project/deep/train.json', 'r') as fp:
    for line in fp:
        data = json.loads(line)
        bio_list.update(data["bio_tags"])

label2index = {x:i for i,x in enumerate(bio_list)}
index2label = {i:x for i,x in enumerate(bio_list)}

print(label2index)
print(index2label)


{'I-Treatment.Drug': 0, 'I-Subject.Sub_Disorder': 1, 'I-Treatment.Treat_Disorder': 2, 'I-Treatment.Time_elapsed': 3, 'I-Subject': 4, 'I-Treatment': 5, 'I-Subject.Gender': 6, 'I-Treatment.Duration': 7, 'I-Treatment.Freq': 8, 'I-Adverse_event.Trigger': 9, 'I-Effect': 10, 'I-Treatment.Route': 11, 'I-Subject.Population': 12, 'I-Potential_therapeutic_event.Trigger': 13, 'I-Subject.Race': 14, 'I-Treatment.Dosage': 15, 'O': 16, 'I-Subject.Age': 17}
{0: 'I-Treatment.Drug', 1: 'I-Subject.Sub_Disorder', 2: 'I-Treatment.Treat_Disorder', 3: 'I-Treatment.Time_elapsed', 4: 'I-Subject', 5: 'I-Treatment', 6: 'I-Subject.Gender', 7: 'I-Treatment.Duration', 8: 'I-Treatment.Freq', 9: 'I-Adverse_event.Trigger', 10: 'I-Effect', 11: 'I-Treatment.Route', 12: 'I-Subject.Population', 13: 'I-Potential_therapeutic_event.Trigger', 14: 'I-Subject.Race', 15: 'I-Treatment.Dosage', 16: 'O', 17: 'I-Subject.Age'}


In [7]:
# build the dataset from JSON files with datasets.load_dataset()
from datasets import load_dataset
dataset = load_dataset("json", data_files = {"train": "/content/drive/MyDrive/Final_Project/deep/train.json", "validation": "/content/drive/MyDrive/Final_Project/deep/dev.json", "test":"/content/drive/MyDrive/Final_Project/deep/test.json"})
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'id', 'bio_tags'],
        num_rows: 2898
    })
    validation: Dataset({
        features: ['sentence', 'id', 'bio_tags'],
        num_rows: 961
    })
    test: Dataset({
        features: ['sentence', 'id', 'bio_tags'],
        num_rows: 968
    })
})


In [8]:
# check whether the dataset is loaded correctly by checking whether the tokens and the IO tags are well aligned
words = dataset["train"][0]["sentence"]
labels = dataset["train"][0]["bio_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    max_length = max(len(word), len(label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += label + " " * (max_length - len(label) + 1)

print(line1)
print(line2)

OBJECTIVE : To test the hypothesis that tumor necrosis factor ( TNF) - alpha may mediate the loss and the dedifferentiation of subcutaneous fat tissue in the insulin          - induced                 lipoatrophies of a diabetic                   patient who presented extensive lesions . 
O         O O  O    O   O          O    O     O        O      O O    O O     O   O       O   O    O   O   O                 O  O            O   O      O  O   I-Treatment.Drug O I-Adverse_event.Trigger I-Effect      O  O I-Treatment.Treat_Disorder O       O   O         O         O       O 


In [10]:
# load model's tokenizer
from transformers import AutoTokenizer

# add variables for model names and output HuggingFace directories for easier shifting between models
model_checkpoint = "allenai/scibert_scivocab_cased"
output_model_name = "scibert_all_deep"
output_model_dir = "jialinselenasong/" + output_model_name

# RoBERTa tokenizer requires and additional argument of add_prefix_space=True
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

In [23]:
inputs = tokenizer(dataset["train"][0]["sentence"], is_split_into_words=True)

In [24]:
inputs.tokens()

['[CLS]',
 'objective',
 ':',
 'to',
 'test',
 'the',
 'hypothesis',
 'that',
 'tumor',
 'necrosis',
 'factor',
 '(',
 'tn',
 '##f',
 ')',
 '-',
 'alpha',
 'may',
 'mediate',
 'the',
 'loss',
 'and',
 'the',
 'ded',
 '##iff',
 '##erentiation',
 'of',
 'subcutaneous',
 'fat',
 'tissue',
 'in',
 'the',
 'insulin',
 '-',
 'induced',
 'lip',
 '##oa',
 '##troph',
 '##ies',
 'of',
 'a',
 'diabetic',
 'patient',
 'who',
 'presented',
 'extensive',
 'lesions',
 '.',
 '[SEP]']

In [22]:
inputs.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 11,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 20,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 30,
 30,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 None]

In [14]:
# results from the tokenizers are longer than the tokens in the dataset
# this function make sure the IO tags matches with the tokens from the newly tokenized text

def align_labels_with_tokens(labels, word_ids):
    label_text = labels
    label_index = [label2index[label] for label in labels]
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # new word
            current_word = word_id
            label = -100 if word_id is None else label_index[word_id]
            new_labels.append(label)
        elif word_id is None:
            # tokens like CLS, PAD, SEP
            new_labels.append(-100)
        else:
            # same word
            label = label_index[word_id]
            new_labels.append(label)

    return new_labels

In [28]:
# check whether the new indices are genearated correctly
# the correct new indices should be longer and have -100 on both ends
labels = dataset["train"][0]["bio_tags"]
word_ids = inputs.word_ids()
print([label2index[label] for label in labels])
print(align_labels_with_tokens(labels, word_ids))

[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 0, 16, 9, 10, 16, 16, 2, 16, 16, 16, 16, 16, 16]
[-100, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 0, 16, 9, 10, 10, 10, 10, 16, 16, 2, 16, 16, 16, 16, 16, 16, -100]


In [ ]:
# align tokens and IO tags for multiple examples so we can use batch
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["sentence"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["bio_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
# align tokens and IO tags by batches
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/2898 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/961 [00:00<?, ? examples/s]

Map:   0%|          | 0/968 [00:00<?, ? examples/s]

In [29]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [30]:
# import the evaluation metric
import evaluate

metric = evaluate.load("seqeval")

In [31]:
# prepare the predicted labels and the true labels for evaluation
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    print(labels)
    print(predictions)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[index2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [index2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [32]:
# load the model
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=index2label,
    label2id=label2index,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# load training arguments
from transformers import TrainingArguments

args = TrainingArguments(
    output_model_name,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
# Training starts here
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
# send the checkpoint to huggingface
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/jialinselenasong/scibert_all_deep/commit/0ca64422d0a99f7438b9c8dd6fe78b913b7ef194', commit_message='Training complete', commit_description='', oid='0ca64422d0a99f7438b9c8dd6fe78b913b7ef194', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# predict a random sentence
from transformers import pipeline

model_checkpoint = output_model_dir
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("Subjects receiving ivacaftor were 55% less likely to have a pulmonary exacerbation than were patients receiving placebo, through week 48 (P<0.001).")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/437M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'Subject',
  'score': 0.7752015,
  'word': 'subjects',
  'start': 0,
  'end': 8},
 {'entity_group': 'Treatment.Drug',
  'score': 0.99969643,
  'word': 'ivacaftor',
  'start': 19,
  'end': 28},
 {'entity_group': 'Adverse_event.Trigger',
  'score': 0.70256454,
  'word': 'have',
  'start': 53,
  'end': 57},
 {'entity_group': 'Effect',
  'score': 0.9996221,
  'word': 'pulmonary exacerbation',
  'start': 60,
  'end': 82},
 {'entity_group': 'Subject',
  'score': 0.84673023,
  'word': 'patients',
  'start': 93,
  'end': 101}]